In [1]:
#%run /home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones_insercion_laboral.ipynb
%run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones_insercion_laboral.ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pickle
warnings.filterwarnings('ignore')

In [2]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'


In [31]:
with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerados.pickle', 'rb') as file:
    dict_cod_aglomerado = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_provincia.pickle', 'rb') as file:
    map_aglomerado_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_region.pickle', 'rb') as file:
    map_aglomerado_region = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia_region.pickle', 'rb') as file:
    map_provincia_region = pickle.load(file)

In [27]:
df_CBT = pd.read_csv(pathdata + 'data_output/Canasta_Basica_Total_Regiones_2016-2022-promedios-moviles.csv', delimiter=',', header=0, index_col=[0])
df_adultos_equiv = pd.read_csv(pathdata + 'data_input/canastas_basicas/adultos_equivalente.csv')

def capitalize_first_letter(s):
    return s.capitalize()

df_CBT = df_CBT.rename(columns=capitalize_first_letter)
df_CBT.rename(columns={'Gran_buenos_aires':'Gran Buenos Aires'}, inplace=True)

In [4]:
# archivos_people_aglomerados = ['personas_T121.txt', 'personas_T221.txt', 
#                                'personas_T321.txt', 'personas_T421.txt', 
#                                'personas_T122.txt', 'personas_T222.txt', 
#                                'personas_T322.txt', 'personas_T422.txt']

# fechas_trimestrales = pd.date_range(start='2021-01-01', end='2022-12-31', freq='QS')
# lista_df_insercion = []

# for archivo in archivos_people_aglomerados:
    
#     df = pd.read_csv(pathdata + 'data_input/' + archivo, delimiter=';', low_memory=False)
    
#     df_insercion = calcular_insercion_laboral(df, dict_cod_aglomerado, tipo='Aglomerado', base='Individual', confidence_level=0.95, pool=False)
#     df_insercion['PROVINCIA'] = df_insercion.index.map(map_aglomerado_provincia)
#     df_insercion_provincias = df_insercion.groupby('PROVINCIA').mean()

#     lista_df_insercion.append(df_insercion_provincias)    

# dict_dfs_insercion_aglomerado = dict(zip(fechas_trimestrales, lista_df_insercion))

In [5]:
# archivos_people_tot_urbano = ['personas_tot_urb_3T_21.txt', 
#                                 'personas_tot_urb_3T_22.txt']

# fechas_tot_urb = pd.date_range(start='2021-07-01', end='2022-12-31', freq='AS-JUL')
# lista_df_insercion = []

# for archivo in archivos_people_tot_urbano:
    
#     df = pd.read_csv(pathdata + 'data_input/' + archivo, delimiter=';', low_memory=False)
    
#     df_insercion = calcular_insercion_laboral(df, dict_cod_provincia, tipo='Urbano', base='Individual', confidence_level=0.95, pool=False)

#     lista_df_insercion.append(df_insercion)    

# dict_dfs_insercion_urbano = dict(zip(fechas_tot_urb, lista_df_insercion))

In [6]:
# with open(pathdata + 'data_output/' + 'diccionario_insercion_laboral_aglomerados.pickle', 'wb') as file:
#     pickle.dump(dict_dfs_insercion_aglomerado, file)

# with open(pathdata + 'data_output/' + 'diccionario_insercion_laboral_urbano.pickle', 'wb') as file:
#     pickle.dump(dict_dfs_insercion_urbano, file)



In [28]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)


In [7]:
df_insercion_pool = calcular_insercion_laboral(df_people_pool, dict_cod_provincia, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, ingreso_salarial_media = \
    indicadores_insercion_laboral(df_people_pool, dict_cod_provincia, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
df_insercion_pool.to_pickle(pathdata + 'data_output/df_insercion_laboral_pool.pickle')

In [29]:
%run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/02_funciones_autonomia_economica.ipynb


In [38]:
tasa, size, ratio, error = poblacion_inactiva_con_ingresos(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
inactivos_con_ingreso = pd.concat([tasa, size, ratio, error], axis=1)

inactivos_con_ingreso.index = inactivos_con_ingreso.index.map(dict_cod_provincia)

In [32]:
hogares_jefatura = ratio_no_pobreza(df_people_pool, df_houses_pool, df_CBT, df_adultos_equiv, dict_cod_provincia, map_provincia_region)

In [44]:
ratio_actividad.to_csv(pathdata + 'data_output/indicadores/01_ratio_actividad.csv')
empleo_asalariado_con_descuento.to_csv(pathdata + 'data_output/indicadores/02_empleo_asalariado_con_descuento.csv')
horas_remuneradas_media.to_csv(pathdata + 'data_output/indicadores/03_horas_promedio_ocup_ppal.csv')
ingreso_salarial_media.to_csv(pathdata + 'data_output/indicadores/04_ingreso_salarial.csv')
inactivos_con_ingreso.to_csv(pathdata + 'data_output/indicadores/06_dependencia_economica.csv')
hogares_jefatura.to_csv(pathdata + 'data_output/indicadores/07_hogares_no-pobres_jefatura.csv')